In [1]:
#LOAD AND DISPLAY IMAGES WITH YOLO ANNOTATIONS
import os
import cv2
import random
import yaml
dataset_path = r"C:\Users\nethr\Downloads\FYP\detect car" 
image_folder_path = os.path.join(dataset_path, 'train', 'images')
label_folder_path = os.path.join(dataset_path, 'train', 'labels')
yaml_path = os.path.join(dataset_path, 'data.yaml')

print(f"Looking for images in: {image_folder_path}")
print(f"Looking for labels in: {label_folder_path}")
print(f"Loading class names from: {yaml_path}")

try:
    with open(yaml_path, 'r') as f:
        data_yaml = yaml.safe_load(f)
    class_names = data_yaml['names'] # ['bus', 'car', 'truck']
    print(f"Successfully loaded class names: {class_names}")
except FileNotFoundError:
    print(f"Error: 'data.yaml' not found at {yaml_path}")
    print("Please make sure the 'dataset_path' variable is correct.")
    raise
except Exception as e:
    print(f"Error loading YAML file: {e}")
    raise

#List and Filter Image Files
try:
    image_files = [f for f in os.listdir(image_folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not image_files:
        raise FileNotFoundError(f"No image files found in {image_folder_path}")
    print(f"Found {len(image_files)} image files.")
except Exception as e:
    print(f"Error listing image files: {e}")
    raise

#Select, Load, Draw, and Display Images
num_images_to_show = 10
random_image_files = random.sample(image_files, min(num_images_to_show, len(image_files)))
print(f"Displaying {len(random_image_files)} random images with annotations...")

for image_name in random_image_files:
    image_path = os.path.join(image_folder_path, image_name)
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Could not load image {image_name}. Skipping.")
        continue
    img_height, img_width = img.shape[:2]
    label_name = os.path.splitext(image_name)[0] + '.txt'
    label_path = os.path.join(label_folder_path, label_name)
    
    if not os.path.exists(label_path):
        print(f"Warning: No label file found for {image_name}. Displaying image without boxes.")
    else:
        #Read and Draw Bounding Boxes
        with open(label_path, 'r') as f:
            for line in f:
                # YOLO format
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Warning: Skipping malformed line in {label_name}: {line}")
                    continue
                class_id, x_center, y_center, w, h = map(float, parts)
                #Convert from 0-1 range to pixel values
                x_center_abs = x_center * img_width
                y_center_abs = y_center * img_height
                w_abs = w * img_width
                h_abs = h * img_height
                #Calculate top-left (x1, y1) and bottom-right (x2, y2)
                x1 = int(x_center_abs - (w_abs / 2))
                y1 = int(y_center_abs - (h_abs / 2))
                x2 = int(x_center_abs + (w_abs / 2))
                y2 = int(y_center_abs + (h_abs / 2))
                #Get class name
                label_text = class_names[int(class_id)]
                # Draw the rectangle
                cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
                # Draw the label text
                cv2.putText(img, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    #Resize and Display
    max_width = 800
    if img_width > max_width:
        ratio = max_width / img_width
        new_height = int(img_height * ratio)
        img_display = cv2.resize(img, (max_width, new_height))
    else:
        img_display = img
    cv2.imshow(image_name, img_display)
    
print("\nPress any key to close the image windows.")
cv2.waitKey(0)
cv2.destroyAllWindows()
print("Image display finished.")

Looking for images in: C:\Users\nethr\Downloads\FYP\detect car\train\images
Looking for labels in: C:\Users\nethr\Downloads\FYP\detect car\train\labels
Loading class names from: C:\Users\nethr\Downloads\FYP\detect car\data.yaml
Successfully loaded class names: ['bus', 'car', 'truck']
Found 17044 image files.
Displaying 10 random images with annotations...

Press any key to close the image windows.
Image display finished.
